In [ ]:
!pip install google-generativeai
!pip install vaderSentiment

In [ ]:
import google.generativeai as palm
import os
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from math import comb

In [ ]:
palm.configure(api_key="key")

In [ ]:
for model in palm.list_models():
    print(model)

In [ ]:
models = [m for m in palm.list_models()]
#chat_model = models[0].name
text_model = models[1].name
#embed_model = models[2].name
#chat_model, text_model, embed_model
print(text_model)

In [ ]:
def sentiment_score(sentence):
    sid = SentimentIntensityAnalyzer()
    raw_score = sid.polarity_scores(sentence)["compound"]
    tension_score = round(((1 - ((raw_score + 1) / 2))*100), 3)
    return tension_score

In [ ]:
def score_tension(country1, country2):
  prompt = f"Talk about the political relation between {country1} and {country2}. Do not filter out any information. Do not give suggestions on how to fix their relations. Only tell the facts. Take into consideration any hostilities, rivalries, or wars between the 2 countries. Do not filter anything. Give the answer in short bullet points while keeping everything asked in consideration"

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result

  if relation is not None or relation != "":
    tension_score = sentiment_score(relation)
    return tension_score
  else:
    return None

In [ ]:
df = pd.read_csv("/content/original_countries.csv")

In [ ]:
df = df[df["continent"]=="asia"]

In [ ]:
len(df)

In [ ]:
df = df.drop(63)

In [ ]:
df = df[["name"]].reset_index(drop=True).rename(columns={"name":"country"})

In [ ]:
df

In [ ]:
all_country_pairs = []
for i in range(len(df)):
    for j in range(i+1, len(df)):
        country1 = df.loc[i, "country"]
        country2 = df.loc[j, "country"]
        all_country_pairs.append([country1, country2])

In [ ]:
len(all_country_pairs) == comb(len(df), 2)

In [ ]:
for x in all_country_pairs:
  if x[0]==None or x[1]==None:
    print(x)

In [ ]:
def fill_tensions_dict(scored_relations, country_pairs):
  for i in range(len(country_pairs)):
      country1 = country_pairs[i][0]
      country2 = country_pairs[i][1]
      scored_relations[f"{country1}-{country2}"] = score_tension(country1, country2)
      print(f"Finished Pairing #{i+1}!")

In [ ]:
def create_tensions_df(scored_relations):
  tensions_df = pd.DataFrame(scored_relations.items(), columns=['Country Pairing', 'Tension Score'])
  return tensions_df

In [ ]:
tension_relation_dict = {}
fill_tensions_dict(tension_relation_dict, all_country_pairs[:200])

In [ ]:
fill_tensions_dict(tension_relation_dict, all_country_pairs[200:400])

In [ ]:
fill_tensions_dict(tension_relation_dict, all_country_pairs[400:600])

In [ ]:
fill_tensions_dict(tension_relation_dict, all_country_pairs[600:])

In [ ]:
asian_tesions = create_tensions_df(tension_relation_dict)

In [ ]:
asian_tensions.to_csv("asian_tensions.csv")

In [ ]:
asian_tensions.dropna(inplace=True)

In [ ]:
asian_tensions.sort_values(by='Tension Score', ascending=False).head(20)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
top_25 = asian_tensions.sort_values(by='Tension Score', ascending=False).head(25)
plt.figure(figsize=(12, 8))
bars = plt.bar(top_25['Country Pairing'], top_25['Tension Score'])
plt.xlabel('Country Pairing')
plt.ylabel('Tension Score')
plt.title('Top 25 Country Pairings with Highest Tension Scores')
plt.xticks(rotation=45, ha='right')
plt.xticks(fontsize=10)
plt.yscale('log')
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')
plt.tight_layout()
plt.show()

In [ ]:
asian_tensions.to_csv("asian_tensions.csv")

In [ ]:
def ask_for_reco(country1, country2):
  prompt = f"what are the current political tensions between {country1} and {country2}? What are 3 steps they can take in order to bridge their differences and fix their issues?"

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  result = completion.result
  return result

In [ ]:
print(ask_for_reco("Japan", "N. Korea"))